<a href="https://colab.research.google.com/github/aSafarpoor/OSN_FAD/blob/main/CLS/FAD%2BNN%2BNode2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install node2vec

In [2]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.4 MB/s eta 0:00:00


In [3]:
import networkx as nx
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from node2vec import Node2Vec
import random
SEED = 10

In [53]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv, GINConv
from sklearn.metrics import roc_curve,accuracy_score, roc_auc_score, confusion_matrix
import torch.nn as nn


# read data

In [5]:
edges = []
with open('graph.txt', 'r') as file:
    for line in file:
        # Split each line into two numbers and convert them to integers
        pair = list(map(int, line.strip().split()))
        edges.append(pair)


with open('train.txt', 'r') as file:
    lines = file.readlines()
    btrain = list(map(float, lines[0].strip().split()))
    strain = list(map(float, lines[1].strip().split()))

with open('test.txt', 'r') as file:
    lines = file.readlines()
    btest = list(map(float, lines[0].strip().split()))
    stest = list(map(float, lines[1].strip().split()))

In [6]:
print("Length of edges:", len(edges),
      "| Length of btrain:", len(btrain),
      "| Length of btest:", len(btest),
      "| Length of strain:", len(strain),
      "| Length of stest:", len(stest))

Length of edges: 6818501 | Length of btrain: 20000 | Length of btest: 158377 | Length of strain: 10000 | Length of stest: 81263


# data preparation

In [7]:
bigG = nx.Graph()
bigG.add_edges_from(edges)

In [8]:
trainG = bigG.subgraph(btrain+strain).copy()
testG = bigG.subgraph(btest+stest).copy()

In [9]:
print(len(bigG.nodes()),len(trainG.nodes()),len(testG.nodes()))

269640 30000 239640


In [10]:
def reindex_graph_and_dictionaries(G, train, test):
    new_index_mapping = {old_index: new_index for new_index, old_index in enumerate(G.nodes())}
    G_reindexed = nx.relabel_nodes(G, new_index_mapping)

    train = [new_index_mapping[int(node)] for node in train]
    test =  [new_index_mapping[int(node)] for node in test]

    return G_reindexed, train, test

In [11]:
trainG,btrain,strain =reindex_graph_and_dictionaries(trainG, btrain,strain)
testG,btest,stest =reindex_graph_and_dictionaries(testG, btest,stest)

In [12]:
trainDictionaryX = {}
trainDictionaryY = {}
testDictionaryX = {}
testDictionaryY = {}

random.seed(SEED)

c=0
for node in btrain:
    if random.random() < 0.20:
        c+=1
        trainDictionaryX[node] = [0]
    else:
        trainDictionaryX[node] = [0.5]
    trainDictionaryY[node] = 0
print(c)

c=0
for node in strain:
    if random.random() < 0.20:
        c+=1
        trainDictionaryX[node] = [1]
    else:
        trainDictionaryX[node] = [0.5]
    trainDictionaryY[node] = 1
print(c)

c=0
for node in btest:
    if random.random() < 0.1:
        c+=1
        testDictionaryX[node] = [0]
    else:
        testDictionaryX[node] = [0.5]
    testDictionaryY[node] = 0
print(c)

c=0
for node in stest:
    if random.random() < 0.1:
        c+=1
        testDictionaryX[node] = [1]
    else:
        testDictionaryX[node] = [0.5]
    testDictionaryY[node] = 1
print(c)

4038
1986
15631
8053


In [ ]:
# print(len(btrain))
# print(len(strain))
# print(len(btest))
# print(len(stest))


#model

In [13]:
# Prepare data (assuming PyTorch Geometric format for graphs)
def prepare_data(graph, dictionaryX, dictionaryY):
    x = torch.tensor([dictionaryX[node] for node in graph.nodes()], dtype=torch.float)
    y = torch.tensor([dictionaryY[node] for node in graph.nodes()], dtype=torch.long)
    # edge_index = torch.tensor(list(graph.edges())).t().contiguous()
    edge_index = torch.tensor(list(graph.edges()), dtype=torch.long).t().contiguous()
    return x, y, edge_index

In [14]:
def pure_sybil_test(test_x, test_y):
    print(len(test_x))
    mask = (test_y == 1)
    x = test_x[mask]
    y = test_y[mask]
    print(len(x))
    return x, y, mask

In [15]:
train_x, train_y, train_edge_index = prepare_data(trainG, trainDictionaryX, trainDictionaryY)
test_x, test_y, test_edge_index = prepare_data(testG, testDictionaryX, testDictionaryY)
test_x2, test_y2, mask2 = pure_sybil_test(test_x, test_y)


239640
81263


## spiker

In [58]:
class InputSpikingLayer(nn.Module):
    def __init__(self, threshold=0.5):
        super(InputSpikingLayer, self).__init__()
        self.threshold = threshold

    def forward(self, x):
        # Apply simple spiking to the input features only
        spikes = (x > self.threshold).float()
        return spikes * x

class GATWithInputSpiking(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, heads=1, threshold=0.5):
        super(GATWithInputSpiking, self).__init__()
        self.spike_input = InputSpikingLayer(threshold=threshold)  # Apply spiking only to the input
        self.conv1 = GATConv(input_dim, hidden_dim, heads=heads)
        self.conv2 = GATConv(hidden_dim * heads, output_dim, heads=1)  # Single head in the final layer

    def forward(self, x, edge_index):
        # Apply spiking to input only
        x = self.spike_input(x)  # Apply spiking only at the input layer
        x = self.conv1(x, edge_index)
        x = F.elu(x)  # ELU activation
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)



## other models

In [16]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout=True):
        super(GCN, self).__init__()
        self.dropout = dropout
        self.convs = torch.nn.ModuleList()
        self.num_classes = output_dim
        for i in range(num_layers):
            if i == 0:  # First layer
                self.convs.append(GCNConv(input_dim, hidden_dim, bias=True))
            elif i == num_layers - 1:  # Last layer
                self.convs.append(GCNConv(hidden_dim, output_dim, bias=True))
            else:  # Middle layers
                self.convs.append(GCNConv(hidden_dim, hidden_dim, bias=True))

    def forward(self, x, edge_index):
        h = x
        i = 0
        for conv in self.convs:
            if self.dropout:
                h = F.dropout(h, p=0.5, training=self.training)
            h = conv(h, edge_index)
            if i < len(self.convs) - 1:
                h = F.tanh(h)
            i += 1

        if self.num_classes == 1:
            return F.sigmoid(h), h
        else:
            return F.softmax(h, dim=1)




class GAT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, heads=1):
        super(GAT, self).__init__()
        self.conv1 = GATConv(input_dim, hidden_dim, heads=heads)
        self.conv2 = GATConv(hidden_dim * heads, output_dim, heads=1)  # Single head in the final layer

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)  # ELU is commonly used in GAT models instead of ReLU
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [62]:
class GINGATHybrid(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, gin_hidden_dim, gat_heads=1):
        super(GINGATHybrid, self).__init__()
        # Define GIN layer with an MLP
        self.gin_mlp = nn.Sequential(
            nn.Linear(input_dim, gin_hidden_dim),
            nn.ReLU(),
            nn.Linear(gin_hidden_dim, hidden_dim)
        )
        self.gin_conv = GINConv(self.gin_mlp)

        # Define GAT layers
        self.gat_conv1 = GATConv(hidden_dim, hidden_dim, heads=gat_heads)
        self.gat_conv2 = GATConv(hidden_dim * gat_heads, output_dim, heads=1)  # Single head in final layer

    def forward(self, x, edge_index):
        # Apply GIN convolution
        x = self.gin_conv(x, edge_index)
        x = F.elu(x)  # Activation after GIN layer

        # Apply GAT layers
        x = self.gat_conv1(x, edge_index)
        x = F.elu(x)  # Activation after GAT layer
        x = self.gat_conv2(x, edge_index)

        # Final output
        return F.log_softmax(x, dim=1)




In [22]:
def run_model(model,epoch_number):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    loss_fn = torch.nn.CrossEntropyLoss()

    model.train()
    for epoch in tqdm(range(epoch_number)):
        optimizer.zero_grad()
        out = model(train_x, train_edge_index)
        loss = loss_fn(out, train_y)
        loss.backward()
        optimizer.step()
        if (epoch+1)%20 == 0 or epoch==0:
            print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

    model.eval()
    with torch.no_grad():
        test_out = model(test_x, test_edge_index)
        pred = test_out.argmax(dim=1).cpu()

        # Compute metrics
        acc = accuracy_score(test_y, pred)
        auc = roc_auc_score(test_y, test_out[:, 1], multi_class='ovo')  # Adjust depending on binary or multi-class
        cm = confusion_matrix(test_y, pred)
        cm = cm.astype('float') / len(pred)


        print(f'Accuracy: {acc:.3f}')
        print(f'AUC: {auc:.3f}')
        print('Confusion Matrix:')
        print(f"                  Predicted Negative    Predicted Positive")
        print(f"Actual Negative       {cm[0, 0]:.3f}           {cm[0, 1]:.3f}")
        print(f"Actual Positive       {cm[1, 0]:.3f}           {cm[1, 1]:.3f}")

        print("\n--------------------------just sybils-----------------------------\n")

        test_out = model(test_x, test_edge_index)
        test_out = test_out[mask2]
        pred = test_out.argmax(dim=1).cpu()
        acc = accuracy_score(test_y2, pred)
        print(f'Accuracy: {acc:.3f}')


    with torch.no_grad():
        test_out = model(test_x, test_edge_index)
        pred = test_out.argmax(dim=1).cpu()

        # Compute AUC score as usual
        auc = roc_auc_score(test_y, test_out[:, 1], multi_class='ovo')
        fpr, tpr, thresholds = roc_curve(test_y, test_out[:, 1])
        j_scores = tpr - fpr
        optimal_threshold = thresholds[j_scores.argmax()]
        pred = (test_out[:, 1] >= optimal_threshold)
        acc = accuracy_score(test_y, pred)

        # just sybil
        test_out = test_out[mask2]
        pred = (test_out[:, 1] >= optimal_threshold)
        acc_sybils = accuracy_score(test_y2, pred)
        print("Accuracy with optimal threshold:", acc)
        print("Accuracy with optimal threshold just for sybils:", acc_sybils)

In [65]:

input_dim = 1
hidden_dim = 16
output_dim = 2
gin_hidden_dim = 32
gat_heads = 4

model = GINGATHybrid(input_dim, hidden_dim, output_dim, gin_hidden_dim, gat_heads)
print(model)
run_model(model,epoch_number=300)

GINGATHybrid(
  (gin_mlp): Sequential(
    (0): Linear(in_features=1, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16, bias=True)
  )
  (gin_conv): GINConv(nn=Sequential(
    (0): Linear(in_features=1, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16, bias=True)
  ))
  (gat_conv1): GATConv(16, 16, heads=4)
  (gat_conv2): GATConv(64, 2, heads=1)
)


  0%|          | 1/300 [00:01<07:25,  1.49s/it]

Epoch 1, Loss: 0.8596888780593872


  7%|▋         | 20/300 [00:12<02:58,  1.57it/s]

Epoch 20, Loss: 0.629577100276947


...

 93%|█████████▎| 280/300 [02:47<00:12,  1.65it/s]

Epoch 280, Loss: 0.4736780822277069


100%|██████████| 300/300 [02:59<00:00,  1.67it/s]

Epoch 300, Loss: 0.5141963958740234


Accuracy: 0.496
AUC: 0.637
Confusion Matrix:
                  Predicted Negative    Predicted Positive
Actual Negative       0.251           0.410
Actual Positive       0.094           0.245

--------------------------just sybils-----------------------------

Accuracy: 0.723
Accuracy with optimal threshold: 0.7091428809881489
Accuracy with optimal threshold just for sybils: 0.4054981972115231


In [39]:
# Example of usage
input_dim = 1
hidden_dim = 16
output_dim = len(set(trainDictionaryY.values()))

model = GATWithInputSpiking(input_dim,hidden_dim,output_dim, heads=4, threshold=0.5)
print(model)
run_model(model,epoch_number=300)

GATWithInputSpiking(
  (spike_input): InputSpikingLayer()
  (conv1): GATConv(1, 16, heads=4)
  (conv2): GATConv(64, 2, heads=1)
)


  0%|          | 1/300 [00:00<04:24,  1.13it/s]

Epoch 1, Loss: 0.7006401419639587


  7%|▋         | 20/300 [00:14<02:27,  1.90it/s]

Epoch 20, Loss: 0.5670578479766846


...

 93%|█████████▎| 280/300 [03:06<00:12,  1.62it/s]

Epoch 280, Loss: 0.5043070912361145


100%|██████████| 300/300 [03:17<00:00,  1.52it/s]

Epoch 300, Loss: 0.5042979121208191


Accuracy: 0.836
AUC: 0.876
Confusion Matrix:
                  Predicted Negative    Predicted Positive
Actual Negative       0.636           0.025
Actual Positive       0.139           0.200

--------------------------just sybils-----------------------------

Accuracy: 0.591
Accuracy with optimal threshold: 0.8624561842764146
Accuracy with optimal threshold just for sybils: 0.7989244797755436


In [43]:
# Example of usage
input_dim = 1
hidden_dim = 32
output_dim = len(set(trainDictionaryY.values()))

model = GATWithInputSpiking(input_dim,hidden_dim,output_dim, heads=4, threshold=0.5)
print(model)
run_model(model,epoch_number=300)

GATWithInputSpiking(
  (spike_input): InputSpikingLayer()
  (conv1): GATConv(1, 32, heads=4)
  (conv2): GATConv(128, 2, heads=1)
)


  0%|          | 1/300 [00:01<08:32,  1.71s/it]

Epoch 1, Loss: 0.6914362907409668


  7%|▋         | 20/300 [00:20<04:33,  1.02it/s]

Epoch 20, Loss: 0.5113051533699036


...

 93%|█████████▎| 280/300 [04:26<00:18,  1.06it/s]

Epoch 280, Loss: 0.504282534122467


100%|██████████| 300/300 [04:44<00:00,  1.05it/s]

Epoch 300, Loss: 0.5042658448219299


Accuracy: 0.835
AUC: 0.875
Confusion Matrix:
                  Predicted Negative    Predicted Positive
Actual Negative       0.637           0.024
Actual Positive       0.142           0.197

--------------------------just sybils-----------------------------

Accuracy: 0.582
Accuracy with optimal threshold: 0.8586838591220164
Accuracy with optimal threshold just for sybils: 0.8040436607065946


In [25]:
# Initialize model, optimizer, and loss function
input_dim = 1
hidden_dim = 16
output_dim = len(set(trainDictionaryY.values()))  # Number of classes

model = GCN(input_dim, hidden_dim, output_dim, num_layers=3)
run_model(model,epoch_number=300)

  1%|          | 2/300 [00:00<00:57,  5.22it/s]

Epoch 1, Loss: 0.7097325325012207


  7%|▋         | 21/300 [00:04<01:02,  4.46it/s]

Epoch 20, Loss: 0.6364270448684692


 14%|█▎        | 41/300 [00:08<00:45,  5.75it/s]

...

Epoch 260, Loss: 0.6174145340919495


 94%|█████████▎| 281/300 [00:51<00:03,  6.07it/s]

Epoch 280, Loss: 0.6271883249282837


100%|██████████| 300/300 [00:55<00:00,  5.44it/s]


Epoch 300, Loss: 0.6209688782691956
Accuracy: 0.661
AUC: 0.402
Confusion Matrix:
                  Predicted Negative    Predicted Positive
Actual Negative       0.661           0.000
Actual Positive       0.339           0.000

--------------------------just sybils-----------------------------

Accuracy: 0.000
Accuracy with optimal threshold: 0.34184610248706393
Accuracy with optimal threshold just for sybils: 0.9961360028549279


In [26]:
input_dim = 1
hidden_dim = 4
output_dim = len(set(trainDictionaryY.values()))  # Number of classes

model = GCN(input_dim, hidden_dim, output_dim, num_layers=4)
run_model(model,epoch_number=300)

  0%|          | 1/300 [00:00<00:39,  7.63it/s]

Epoch 1, Loss: 0.7020566463470459


  7%|▋         | 21/300 [00:02<00:37,  7.35it/s]

Epoch 20, Loss: 0.6369262337684631


 14%|█▎        | 41/300 [00:05<00:35,  7.21it/s]

...

Epoch 260, Loss: 0.6267896890640259


 94%|█████████▎| 281/300 [00:43<00:02,  7.34it/s]

Epoch 280, Loss: 0.6267450451850891


100%|██████████| 300/300 [00:46<00:00,  6.52it/s]


Epoch 300, Loss: 0.6215488910675049
Accuracy: 0.661
AUC: 0.445
Confusion Matrix:
                  Predicted Negative    Predicted Positive
Actual Negative       0.661           0.000
Actual Positive       0.339           0.000

--------------------------just sybils-----------------------------

Accuracy: 0.000
Accuracy with optimal threshold: 0.3431772658988483
Accuracy with optimal threshold just for sybils: 0.9957545254297774


In [27]:
input_dim = 1
hidden_dim = 16
output_dim = len(set(trainDictionaryY.values()))  # Number of classes

model = GAT(input_dim, hidden_dim, output_dim, heads=4)
run_model(model,epoch_number=300)

  0%|          | 1/300 [00:00<02:37,  1.90it/s]

Epoch 1, Loss: 0.687141478061676


  7%|▋         | 20/300 [00:10<02:59,  1.56it/s]

Epoch 20, Loss: 0.6101386547088623


 13%|█▎        | 40/300 [00:21<02:17,  1.89it/s]

...

 93%|█████████▎| 280/300 [03:02<00:10,  1.87it/s]

Epoch 280, Loss: 0.407958984375


100%|██████████| 300/300 [03:20<00:00,  1.50it/s]

Epoch 300, Loss: 0.4067078232765198


Accuracy: 0.858
AUC: 0.896
Confusion Matrix:
                  Predicted Negative    Predicted Positive
Actual Negative       0.614           0.047
Actual Positive       0.095           0.244

--------------------------just sybils-----------------------------

Accuracy: 0.720
Accuracy with optimal threshold: 0.8569979969954933
Accuracy with optimal threshold just for sybils: 0.7242410445098015


In [28]:
input_dim = 1
hidden_dim = 4
output_dim = len(set(trainDictionaryY.values()))  # Number of classes

model = GAT(input_dim, hidden_dim, output_dim, heads=4)
run_model(model,epoch_number=300)

  0%|          | 1/300 [00:00<01:12,  4.10it/s]

Epoch 1, Loss: 0.7162822484970093


  7%|▋         | 20/300 [00:04<01:07,  4.18it/s]

Epoch 20, Loss: 0.6333376169204712


 13%|█▎        | 40/300 [00:10<01:08,  3.81it/s]

...

 93%|█████████▎| 280/300 [01:14<00:04,  4.26it/s]

Epoch 280, Loss: 0.4124368131160736


100%|██████████| 300/300 [01:21<00:00,  3.70it/s]

Epoch 300, Loss: 0.41146260499954224


Accuracy: 0.855
AUC: 0.895
Confusion Matrix:
                  Predicted Negative    Predicted Positive
Actual Negative       0.611           0.050
Actual Positive       0.094           0.245

--------------------------just sybils-----------------------------

Accuracy: 0.721
Accuracy with optimal threshold: 0.8555291270238692
Accuracy with optimal threshold just for sybils: 0.7221859887033459
